# Strumieniowa bezstratna kompresja słownikowa, Lempel-Ziv

_Na podstawie:_ [_The Beauty of Lempel-Ziv Compression_](https://youtu.be/RV5aUr8sZD0)

**Zadanie:**   
Napisać modelowy przykład algorytmu kompresji metodą Lempel-Ziv.

Należy zaimplementować dwie funkcje:  
* `encode` - przyjmuje ciąg znaków i zwraca jego skompresowaną wersję (w postaci listy krotek)
* `decode` - przyjmuje skompresowaną wiadomość i zwraca jej oryginalną treść (zadanie za dwa plusy :).

Algorytm jest pokazany na filmiku od 5:20. Polega na wczytywaniu wiadomości znak po znaku i kompresji wiadomości na podstawie słownika fraz, który jest tworzony na bieżąco.

W trakcie rozwiązywania tego zadania pojawią się pewne problemy i przypadki, które trzeba będzie obsłużyć w jakiś sposób. Przykład z filmiku w sprytny sposób je unika ;)

**Termin oddania:** 27.01.2019 20:00:00

Kilka rzeczy, które mogą się przydać:
* Do tworzenia słownika fraz, może się przydać... słownik (`{key: value, key2: value}`). Jest on również przydatny z tego względu, że wyszukiwanie danej frazy ma stałą [złożoność czasową](https://wiki.python.org/moin/TimeComplexity) (_Get Item_).
* Iterator

```
import itertools

counter = itertools.count(1)

print(next(counter))
print(next(counter))
print(next(counter))
```
* Ciąg znaków można traktować podobnie jak listę: `'abc'[:-2]`
* Sprawdzanie czy klucz istnieje: `key in dictionary`

** Solution **

In [47]:
import itertools

def encode(message):
    
    dict_counter = itertools.count(1)
    
    dictionary = dict()
    compressed_message = list()
    
    current = "" #current processed substring
    current_code = None  #index of current processed substring
    
    for index, char in enumerate(message):
        
        if current+char in dictionary:
            #if the current+char exists in dictionary, look for longer substring
            
            current += char
            
            if index == len(message)-1:
                #there is no characters left in the message
                #so the current must be included in the compressed message
                
                current_code = dictionary.get(current)
                compressed_message.append((current_code,""))
        
        else:
            
            if current == "":
                #the first occurence of the character
            
                dictionary[char] = next(dict_counter)
            
            else:
            
                current_code = dictionary.get(current)
                dictionary[current+char] = next(dict_counter)
                
            compressed_message.append((current_code,char))
            current_code = None
            current="" 
    
    #return (compressed_message,dictionary)
    return compressed_message
    
def decode(compressed_message):
    
    dict_counter = itertools.count(1)
    dictionary = dict()
    message=""
    
    for code, char in compressed_message:
        #print([code,char])
        if code is None:
            #the first occurence of the char
            dictionary[next(dict_counter)] = char
            message += char
        
        else:
            if char == '':
                #the last substring in the message
                message += dictionary.get(code)
            
            else:
                message += dictionary.get(code) + char
                dictionary[next(dict_counter)] = dictionary.get(code) + char
    
    return message

** Check **

In [48]:
message = 'AABABBABBAABA'
compressed_message = encode(message)
print(compressed_message)  # [(None, 'A'), (1, 'B'), (2, 'B'), (3, 'A'), (2, 'A')]

decode(compressed_message) == message  # True

[(None, 'A'), (1, 'B'), (2, 'B'), (3, 'A'), (2, 'A')]


True

In [49]:
message = 'banana'
compressed_message = encode(message)
print(compressed_message)  # [(None, 'b'), (None, 'a'), (None, 'n'), (2, 'n'), (2, '')]

decode(compressed_message) == message  # True

[(None, 'b'), (None, 'a'), (None, 'n'), (2, 'n'), (2, '')]


True

In [50]:
message = 'AABABA'
compressed_message = encode(message)
print(compressed_message)  # [(None, 'A'), (1, 'B'), (2, 'A')]

decode(compressed_message) == message  # True

[(None, 'A'), (1, 'B'), (2, 'A')]


True

In [51]:
message = 'ABABCAB'
compressed_message = encode(message)
print(compressed_message)  # [(None, 'A'), (None, 'B'), (1, 'B'), (None, 'C'), (3, '')]

decode(compressed_message) == message  # True

[(None, 'A'), (None, 'B'), (1, 'B'), (None, 'C'), (3, '')]


True

**Beware the Wikipedia**

Wrong solution from: https://pl.wikipedia.org/wiki/LZ78

In [6]:
def LZ78_encode(data):
	D = {}
	n = 1
	c = ''
	result = []
	for s in data:
		if c + s not in D:
			if c == '':
				# specjalny przypadek: symbol 's'
				# nie występuje jeszcze w słowniku
				result.append( (0, s) )
				D[s] = n
			else:
				# ciąg 'c' jest w słowniku
				result.append( (D[c], s) )
				D[c + s] = n
			n = n + 1
			c = ''
		else:
			c = c + s

	return result

def LZ78_decode(data):
	D = {}
	n = 1

	result = []
	for i, s in data:
		if i == 0:
			result.append(s)
			D[n] = s
			n = n + 1
		else:
			result.append(D[i] + s)
			D[n] = D[i] + s
			n = n + 1

	return ''.join(result)

In [45]:
message = 'banana'
compressed_message = LZ78_encode(message)
print(compressed_message)

print(LZ78_decode(compressed_message))
LZ78_decode(compressed_message) == message  # True

[(0, 'b'), (0, 'a'), (0, 'n'), (2, 'n')]
banan


False

In [46]:
message = 'ABABCAB'
compressed_message = LZ78_encode(message)
print(compressed_message)

print(LZ78_decode(compressed_message))
LZ78_decode(compressed_message) == message  # True

[(0, 'A'), (0, 'B'), (1, 'B'), (0, 'C')]
ABABC


False